In [3]:
import pandas as pd
import json
import os
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 1 - CARGA DOS DADOS
# Usamos o arquivo gerado pelo novo Script 01
print("Lendo dados preparados de 2025...")
df_full = pd.read_csv('../dataset/well_breathe_2025_full.csv')

# 2 - PREPARAÇÃO PARA CLUSTERIZAÇÃO
# Criamos as médias anuais para cada cidade usando o novo conjunto de 6 variáveis
features = [
    'temperature_celsius', 
    'humidity', 
    'wind_kph', 
    'precip_mm', 
    'visibility_km', 
    'air_quality_PM2.5'
]

df_cluster = df_full.groupby('location_name')[features].mean().reset_index()

# 3 - NORMALIZAÇÃO (Z-Score)
# Como Visibilidade (km) e Chuva (mm) têm escalas diferentes, a normalização é obrigatória
scaler = StandardScaler()
features_scaled = scaler.fit_transform(df_cluster[features])

# 4 - APLICANDO O K-MEANS
# Vamos utilizar 4 grupos para capturar melhor a diversidade global
print("Executando agrupamento (K-Means)...")
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df_cluster['cluster_id'] = kmeans.fit_predict(features_scaled)

# 5 - DEFINIÇÃO DE PERFIS (LOGICA DE IA)
# Analisamos os centros dos clusters para dar nomes científicos aos grupos
def define_profile(row):
    if row['air_quality_PM2.5'] > 30:
        return "Zona Industrial com Alta Retencao"
    elif row['precip_mm'] > 3.0:
        return "Clima Tropical com Lavagem Atmosferica"
    elif row['visibility_km'] > 9.5 and row['air_quality_PM2.5'] < 15:
        return "Ambiente de Alta Visibilidade e Pureza"
    elif row['wind_kph'] > 18:
        return "Regiao de Alta Dispersao Eolica"
    else:
        return "Clima Temperado Equilibrado"

df_cluster['cluster_name'] = df_cluster.apply(define_profile, axis=1)

# 6 - ATUALIZAÇÃO DO JSON DO APP
print("Integrando perfis ao JSON do aplicativo...")
json_path = '../../mobile-app/assets/data/well_breathe_cities.json'

with open(json_path, 'r', encoding='utf-8') as f:
    cities_data = json.load(f)

for city in cities_data:
    city_name = city['location_name']
    # Busca o perfil gerado para esta cidade
    match = df_cluster[df_cluster['location_name'] == city_name]
    if not match.empty:
        city['cluster_name'] = match.iloc[0]['cluster_name']
    else:
        city['cluster_name'] = "Perfil Nao Identificado"

# 7 - SALVAMENTO FINAL
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(cities_data, f, indent=2, ensure_ascii=False)

print(f"Sucesso! {len(df_cluster)} cidades categorizadas em {len(df_cluster['cluster_name'].unique())} perfis.")
print(f"Perfis identificados: {df_cluster['cluster_name'].unique().tolist()}")

Lendo dados preparados de 2025...
Executando agrupamento (K-Means)...
Integrando perfis ao JSON do aplicativo...
Sucesso! 209 cidades categorizadas em 4 perfis.
Perfis identificados: ['Clima Temperado Equilibrado', 'Regiao de Alta Dispersao Eolica', 'Zona Industrial com Alta Retencao', 'Ambiente de Alta Visibilidade e Pureza']
